In [1]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import torch
import random
from scipy.sparse import csgraph
import sys
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import copy
from utils import *
from graphConvolution import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def aug_normalized_adjacency(adj):
   adj = adj + sp.eye(adj.shape[0])
   adj = sp.coo_matrix(adj)
   row_sum = np.array(adj.sum(1))
   d_inv_sqrt = np.power(row_sum, -0.5).flatten()
   d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
   d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
   return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()


def aug_random_walk(adj):
   adj = adj + sp.eye(adj.shape[0])
   adj = sp.coo_matrix(adj)
   row_sum = np.array(adj.sum(1))
   d_inv = np.power(row_sum, -1.0).flatten()
   d_mat = sp.diags(d_inv)
   return (d_mat.dot(adj)).tocoo()

In [3]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(dataset="pubmed")


[STEP 1]: Upload pubmed dataset.
| # of nodes : 19717
| # of edges : 44325.5
| # of features : 500
| # of clases   : 3
| # of train set : 60
| # of val set   : 500
| # of test set  : 1000


In [4]:
adj = aug_normalized_adjacency(adj)
adj = sparse_mx_to_torch_sparse_tensor(adj).float().cuda()

In [5]:
k = 50 # steps
epoches = 200
num_node =features.shape[0]
feature_list = [features.cuda()]
for i in range(k):
    propagated_fea = torch.spmm(adj,feature_list[-1]).cuda()
    feature_list.append(propagated_fea)

In [6]:
labels = labels.cuda()
idx_train = idx_train.cuda()
idx_val = idx_val.cuda()
idx_test = idx_test.cuda()

### GMLP

In [7]:

def train(epoch, model,record,feature_list):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output_att, output_concat = model(feature_list)
    L1 = F.nll_loss(output_att[idx_train], labels[idx_train])
    L2 = F.nll_loss(output_concat[idx_train], labels[idx_train])
    loss_train = L1 + np.cos(np.pi*epoch/(2*epoches))*L2
    acc_train = accuracy(output_att[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    model.eval()
    output_att, output_concat = model(feature_list)

    loss_val = F.nll_loss(output_att[idx_val], labels[idx_val])
    acc_val = accuracy(output_att[idx_val], labels[idx_val])
    
    loss_test = F.nll_loss(output_att[idx_test], labels[idx_test])
    acc_test = accuracy(output_att[idx_test], labels[idx_test])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'acc_test: {:.4f}'.format(acc_test.item()),
          'time: {:.4f}s'.format(time.time() - t))
    record[acc_val.item()] = acc_test.item()

In [8]:

import torch.nn as nn
import torch.nn.functional as F


class GMLP(nn.Module):
    def __init__(self, nfeat,nhid, nclass, dropout):
        super(GMLP, self).__init__()
        
        self.lr_left1 = nn.Linear((k+1)*features.shape[1], nhid)
        self.lr_left2 = nn.Linear(nhid, nclass)
        self.lr_att = nn.Linear(nhid+nfeat, 1)

        self.lr_right1 = nn.Linear(nfeat, nhid)
        self.lr_right2 = nn.Linear(nhid, nclass)
        
        self.dropout = dropout

    def forward(self,feature_list):     
        drop_features = [F.dropout(feature, self.dropout , training=self.training) for feature in feature_list]
        concat_features = torch.cat(drop_features,dim=1)

        left_1 = F.relu(self.lr_left1(concat_features))
        left_1 = F.dropout(left_1, self.dropout, training=self.training)
        left_2 = self.lr_left2(left_1)
        
        attention_scores = [torch.sigmoid(self.lr_att(torch.cat((left_1,x), dim=1))).view(num_node,1) for x in drop_features]
        W = torch.cat(attention_scores, dim=1)
        W = F.softmax(W,1)
        
        right_1 = torch.mul(drop_features[0], W[:,0].view(num_node,1)) 
        for i in range(1,k):
            right_1 = right_1 + torch.mul(drop_features[i], W[:,i].view(num_node,1)) 
            
        right_1 = F.relu(self.lr_right1(right_1))
        right_1 = F.dropout(right_1, self.dropout, training=self.training)
        
        right_2 = self.lr_right2(right_1)
        
        return F.log_softmax(right_2, dim=1),F.log_softmax(left_2,dim=1)

In [9]:
model = GMLP(nfeat=features.shape[1],nhid =128,
        nclass=labels.max().item() + 1,
        dropout=0.3)
model.cuda()
optimizer = optim.Adam(model.parameters(),lr=0.1, weight_decay=5e-4)

t_total = time.time()
record = {}
for epoch in range(epoches):
    train(epoch,model,record,feature_list)
        
        
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

Epoch: 0001 loss_train: 2.1986 acc_train: 0.3333 acc_val: 0.4160 acc_test: 0.4070 time: 0.4837s
Epoch: 0002 loss_train: 2.0278 acc_train: 0.3667 acc_val: 0.1960 acc_test: 0.1800 time: 0.1957s
Epoch: 0003 loss_train: 10.0510 acc_train: 0.3500 acc_val: 0.4400 acc_test: 0.4600 time: 0.1956s
Epoch: 0004 loss_train: 4.4637 acc_train: 0.4000 acc_val: 0.4160 acc_test: 0.4070 time: 0.1975s
Epoch: 0005 loss_train: 7.9347 acc_train: 0.3333 acc_val: 0.4820 acc_test: 0.4630 time: 0.1981s
Epoch: 0006 loss_train: 3.3514 acc_train: 0.4500 acc_val: 0.4500 acc_test: 0.4710 time: 0.1966s
Epoch: 0007 loss_train: 2.1732 acc_train: 0.4667 acc_val: 0.5380 acc_test: 0.5350 time: 0.1968s
Epoch: 0008 loss_train: 1.3744 acc_train: 0.5667 acc_val: 0.5740 acc_test: 0.5820 time: 0.1976s
Epoch: 0009 loss_train: 1.6961 acc_train: 0.6500 acc_val: 0.6880 acc_test: 0.6730 time: 0.1963s
Epoch: 0010 loss_train: 1.4667 acc_train: 0.7167 acc_val: 0.6460 acc_test: 0.6430 time: 0.1957s
Epoch: 0011 loss_train: 2.7470 acc_trai

Epoch: 0087 loss_train: 0.5302 acc_train: 0.8833 acc_val: 0.7940 acc_test: 0.7950 time: 0.1958s
Epoch: 0088 loss_train: 0.5974 acc_train: 0.9167 acc_val: 0.7620 acc_test: 0.7710 time: 0.1962s
Epoch: 0089 loss_train: 0.6073 acc_train: 0.8500 acc_val: 0.7940 acc_test: 0.7830 time: 0.1963s
Epoch: 0090 loss_train: 0.5588 acc_train: 0.9167 acc_val: 0.7820 acc_test: 0.7630 time: 0.1968s
Epoch: 0091 loss_train: 0.5610 acc_train: 0.9333 acc_val: 0.7920 acc_test: 0.7800 time: 0.1964s
Epoch: 0092 loss_train: 0.5965 acc_train: 0.8667 acc_val: 0.7980 acc_test: 0.7860 time: 0.1972s
Epoch: 0093 loss_train: 0.8312 acc_train: 0.9000 acc_val: 0.7660 acc_test: 0.7660 time: 0.1970s
Epoch: 0094 loss_train: 0.5974 acc_train: 0.8500 acc_val: 0.7500 acc_test: 0.7430 time: 0.1960s
Epoch: 0095 loss_train: 0.6925 acc_train: 0.8833 acc_val: 0.7840 acc_test: 0.7690 time: 0.1976s
Epoch: 0096 loss_train: 0.5530 acc_train: 0.8667 acc_val: 0.8020 acc_test: 0.7800 time: 0.1968s
Epoch: 0097 loss_train: 0.5796 acc_train

Epoch: 0173 loss_train: 0.3586 acc_train: 0.9333 acc_val: 0.7860 acc_test: 0.7850 time: 0.1964s
Epoch: 0174 loss_train: 0.3507 acc_train: 0.8833 acc_val: 0.7880 acc_test: 0.7700 time: 0.1982s
Epoch: 0175 loss_train: 0.4069 acc_train: 0.9000 acc_val: 0.7580 acc_test: 0.7360 time: 0.1976s
Epoch: 0176 loss_train: 0.3919 acc_train: 0.8667 acc_val: 0.8020 acc_test: 0.7930 time: 0.1978s
Epoch: 0177 loss_train: 0.3977 acc_train: 0.8667 acc_val: 0.7940 acc_test: 0.7860 time: 0.1978s
Epoch: 0178 loss_train: 0.3764 acc_train: 0.9000 acc_val: 0.7840 acc_test: 0.7740 time: 0.1965s
Epoch: 0179 loss_train: 0.3120 acc_train: 0.9500 acc_val: 0.7560 acc_test: 0.7450 time: 0.1960s
Epoch: 0180 loss_train: 0.4450 acc_train: 0.8667 acc_val: 0.8200 acc_test: 0.8070 time: 0.1976s
Epoch: 0181 loss_train: 0.3389 acc_train: 0.9333 acc_val: 0.8040 acc_test: 0.7990 time: 0.1982s
Epoch: 0182 loss_train: 0.4093 acc_train: 0.8833 acc_val: 0.7640 acc_test: 0.7560 time: 0.1968s
Epoch: 0183 loss_train: 0.3324 acc_train

### Even with 50 layers, GMLP still keeps a good performance!